In [1]:
import sys
import os
sys.path.append(os.path.abspath('../')) ## needed to import the function.py file
from functions import *
import pandas as pd

2023-03-14 11:50:31.831472: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [ ]:
# specify the path of the Excel file
file_path = '../../exports/tables/Themencluster_Fachexperten_BERTTopic_paraphrase-multilingual-MiniLM-L12-v2_Themencluster_Fachexperten.xlsx'
sheet_name = 'Cluster_L2'  # sheet name

# import the Excel sheet as a pandas DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)
df_export = pd.read_excel(file_path, sheet_name='export')

In [ ]:
## Import docs
filelocation = '../../data/DataTextTrain'
data = pd.read_feather(filelocation)
data = data
docs = data["Kommentar"].values
timestamps = data.yearquarter.to_list()

## Modelvariante 1: Merged Topic Model

In [ ]:
# Load the Original Topic Model
topic_model = BERTopic.load("../../models/BERTTopic_paraphrase-multilingual-MiniLM-L12-v2.model")

In [ ]:
df_print = df[df['Themencluster']!="Outlier"].sort_values(by='Anzahl Kommentare',ascending=False) # Filter out Sonstige

In [ ]:
import plotly.express as px
import plotly.io as pio

fig = px.bar(df_print.sort_values('Anzahl Kommentare', ascending=True),
    y='Themencluster', 
    x='Anzahl Kommentare',
    opacity = 1,
    color_discrete_sequence =color_discrete_sequence
                 )

fig.update_layout(
    title="Die 18 finalen Themencluster und Anzahl Kommentare pro Cluster",
    xaxis_title='',
    yaxis_title='',
    xaxis=dict(tickangle=90),
    width=900, 
    height=450,
    legend_title="")

fig.show()

pio.write_image(fig,"../../exports/images/fig-18-Themencluster-expertenreview.svg") #save fig as svg

In [ ]:
# generate list for topic merge for new aggregated model
grouped = df_export.groupby('cluster_level_2')['Topic'].agg(list).reset_index()
topics_merge_list = grouped.Topic.tolist()

In [ ]:
# Merge Topics based on clusters
topic_model.merge_topics(docs, topics_merge_list)

In [ ]:
# Assign Custom Labels based on Clustering to the Topics
df_new_model = topic_model.get_topic_info()
df_new_lables = df_new_model.merge(df,left_on='Count',right_on='Anzahl Kommentare') # Joinin df to have new Topic ids and Custom cluster namens
df_new_lables.set_index('Topic', inplace=True) # Set the 'Topic' column as the index of the DataFrame
id_mapping_dict = df_new_lables['Themencluster'].to_dict() # Create a dictionary using the index and 'Themencluster' column
topic_model.set_topic_labels(id_mapping_dict) # Assign Lables to model
df_export = topic_model.get_topic_info()

In [ ]:
# Save the Model
topic_model.save('../../models/BERTTopic_paraphrase-multilingual-MiniLM-L12-v2_merged.model')
df_export.to_excel('../../exports/tables/merged_topics.xlsx', index=False)

In [ ]:
df_export

,Topic,Count,Name,CustomName
0,-1,18072,-1_zug_züge_klasse_app,Outlier
1,0,4224,0_zug_verspätung_minuten_verspätungen,Pünktlichkeit
2,1,2922,1_klasse_wagen_sitzplätze_stosszeiten,Platzangebot Zug
3,2,2400,2_teuer_preise_ga_preis,Preis-Leistung
4,3,1785,3_wc_toiletten_wcs_toilette,WC Verfügbarkeit und Sauberkeit
5,4,1663,4_app_ticket_lösen_mobile,Vertrieb und Services
6,5,1655,5_postauto_anschluss_minuten_busse,Fahrplanangebot
7,6,1642,6_hund_kinderwagen_perron_hunde,Sonstiges
8,7,1579,7_maskenpflicht_maske_masken_tragen,Corona
9,8,1435,8_zufrieden_super_personal_job,Lob


## Modelvariante 2: Merged Topic Model with Outlier Reduction

In [ ]:
# # Save outliers reduced model for evaluation comparison
# topics, probs = topic_model.transform(docs)
# new_topics = topic_model.reduce_outliers(docs, topics) # Reduce outliers
# topic_model.update_topics(docs, topics=new_topics) # update Model
# topic_model.save('../../models/BERTTopic_paraphrase-multilingual-MiniLM-L12-v2_merged_oulier_reduced.model') # Save Model

In [ ]:
# Load the Outlier reduced Model
topic_model_outlier_reduced = BERTopic.load("../../models/BERTTopic_paraphrase-multilingual-MiniLM-L12-v2_merged_oulier_reduced.model")

In [ ]:
df_2 = topic_model_outlier_reduced.get_topic_info()
df_print_2 = df_2[df_2['CustomName']!="Outlier"].sort_values(by='Count',ascending=False) # Filter out Sonstige

In [ ]:
import plotly.express as px
import plotly.io as pio

fig = px.bar(df_print_2.sort_values('Count', ascending=True),
    y='CustomName', 
    x='Count',
    opacity = 1,
    color_discrete_sequence =color_discrete_sequence
                 )

fig.update_layout(
    title="Die 18 finalen Themencluster und Anzahl Kommentare pro Cluster",
    xaxis_title='',
    yaxis_title='',
    xaxis=dict(tickangle=90),
    width=900, 
    height=450,
    legend_title="")

fig.show()

pio.write_image(fig,"../../exports/images/fig-18-Themencluster-expertenreview_outlier_reduced.svg") #save fig as svg